<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

import re
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):   
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [ ]:
VERSION    = 10
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TRAIN2  = '../input/all-auto-ru-14-11-2020csv/' # подключил к ноутбуку внешний датасет
DIR_TRAIN3  = '../input/auto-ru-2006-2010/' # подключил к ноутбуку внешний датасет

DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.30   # 20%

# Data

In [ ]:
!ls '../input'

In [ ]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
train2 = pd.read_csv(DIR_TRAIN2+'all_auto_ru_14_11_2020.csv') # датасет для обучения модели
train3 = pd.read_csv(DIR_TRAIN3+'_auto_ru_2006-2010.csv') # датасет для обучения модели

test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train2.head()

In [ ]:
train3.head()

In [ ]:
train2.shape

In [ ]:
train3.shape

In [ ]:
train.head(5)

In [ ]:
train = pd.concat([train, train2,train3])

In [ ]:
train.head()

In [ ]:
train = train.drop_duplicates()

In [ ]:
train.info()

In [ ]:
test.head(5)

In [ ]:
test.info()

In [ ]:
df_train = train.copy()
df_test = test.copy()
df_test['price'] = 0

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

In [ ]:
data = df_test.append(df_train, sort=False) # объединяем

## Data Preprocessing

In [ ]:
data.info()

In [ ]:
data.corr()

нам не потребуются:
'hidden' - неизвестная пустая строка,

'car_url' - ссылка на объявление,

'image' - фото авто, 

'sell_id' - id объявления,

'modelDate' - время появления модели, т.к. есть время производства авто,

'Таможня' - все авто растоможены,

'Состояние' - все авто не требуют ремонта,

'model_info' - расшифровка нас не интересует, а ходовые качества указаны в других полях,

'price_EUR' и 'price_USD' - нас интересуют цены только в рублях

'complectation_dict' - недостаточно времени для обработки

'description' - недостаточно времени для обработки

'equipment_dict' - недостаточно времени для обработки

'priceCurrency' - единственное значение - RUB

'super_gen' - 
'vehicleConfiguration', 'name' - информация повторяется в других колонках


In [ ]:
data = data.drop(['hidden', 'car_url','image', 'sell_id','modelDate', 'price_EUR', 'price_USD',
                  'Таможня', 'Состояние', 'model_info','parsing_unixtime', 'complectation_dict','description', 'equipment_dict',
                  'priceCurrency','super_gen','Комплектация','start_date','Владение','vehicleConfiguration','name'],axis = 1)

In [ ]:
for i in data.columns:
    print(i, len(data[i].value_counts()))

In [ ]:
for i in data.columns:
    print(i, len(data[data[i].isna()]))

упростим bodyType до базовых типов

In [ ]:
data['bodyType'] = data['bodyType'].apply(lambda x: str(x).lower())
data['bodyType'] = data['bodyType'].apply(lambda x: re.findall(r'\w+', x)[0])

In [ ]:
data = data[data['bodyType'] !='nan']

In [ ]:
data['bodyType'].value_counts()

In [ ]:
other_cars = [ 'кабриолет', 'родстер', 'фургон','микровэн', 'лимузин', 'тарга', 'фастбек']
data['bodyType'] = data['bodyType'].apply(lambda x: 'другое' if x in other_cars else x )
data['bodyType'] = data['bodyType'].astype('category')

In [ ]:
data['bodyType'].value_counts().plot(kind='bar')

In [ ]:
data['log_price'] = np.log(data.price)
plt.figure(figsize=(8, 5))
g1 = sns.boxplot(y='log_price', x='bodyType', 
                  data=data, color='darkgreen')
g1.set_title("price of bodyTypes", fontsize=20)

g1.set_ylabel("log_price", fontsize=15)
g1.set_xticklabels(g1.get_xticklabels(),rotation=45)
plt.show()

In [ ]:
data = data[data['numberOfDoors'].notna()]
data['numberOfDoors'] = data['numberOfDoors'].apply(lambda x: 2 if x == 0 else x)

In [ ]:
data['numberOfDoors'].value_counts().plot(kind='bar')

In [ ]:
data['color'].value_counts()

In [ ]:
colors = {
'040001':   'черный',
'FAFBFB':          'белый',
'97948F':          'серый',
'CACECB':          'серебристый',
'0000CC':         'синий',
'EE1D19':         'красный',
'200204':  'бордовый',
'007F00':   'зелёный',
'C49648':   'коричневый',
'22A0F8':         'голубой',
'DEA522':         'золотистый',
'FFD600':          'жёлтый',
'660099':        'пурпурный',
'4A2197':          'фиолетовый',
'FF8649':        'бежевый',
'FFC0CB':  'розовый',
}

In [ ]:
data['color'] = data['color'].replace(colors).apply(lambda x: x.lower())
data['color'] = data['color'].astype("category")

In [ ]:
data['color'].value_counts().plot(kind='bar')

In [ ]:
data = data[data['fuelType'].notna()]
data['fuelType'] = data['fuelType'].astype("category")

In [ ]:
data['fuelType'].value_counts().plot(kind='bar')

In [ ]:
data[data['mileage'].isna()]

In [ ]:
data = data[data['mileage'].notna()]
data['mileage'] = data['mileage'].apply(lambda x: 1000 if x < 1000 else x)
data['mileage'].describe()

In [ ]:
np.log(data['mileage']).hist(bins=77)

In [ ]:
data[data['mileage'] < 1500]['productionDate'].value_counts()

можно разбить все ТС с пробегом менее чем 1500км на:
можно сказать, что автомобили 2020 и 2019 года - новые
остальные - не используемые

In [ ]:
data.loc[(data['mileage'] < 1500) & (data['productionDate'] > 2018 ), 'new_car'] = 1
data['new_car'] = data['new_car'].fillna(0)
data.loc[(data['mileage'] < 1500) & (data['productionDate'] <= 2018 ), 'dont_use_car'] = 1
data['dont_use_car'] = data['dont_use_car'].fillna(0)
data.head()

In [ ]:
data['model_name'].value_counts()

In [ ]:
data['model_name'] = data['model_name'].fillna(data['model'])

In [ ]:
data['model_name'].value_counts()

In [ ]:
brand_list = list(data['brand'].value_counts()[:15].keys())
data['brand'] = data['brand'].apply(lambda x: x if x in brand_list else 'OTHER')
data['brand'] = data['brand'].astype('category')
data['brand'].value_counts().plot(kind='bar')


In [ ]:
engine_list = list(data['engineDisplacement'].value_counts()[:30].keys())
data['engineDisplacement'] = data['engineDisplacement'].apply(lambda x: x if x in engine_list else 'OTHER')
data['engineDisplacement'].value_counts().plot(kind='bar')

In [ ]:
data['enginePower'].value_counts()

In [ ]:
data['enginePower'] = data['enginePower'].apply(lambda x: int(re.findall('(\d+)', str(x))[0]))
data['enginePower'] = data['enginePower'].astype('int')


In [ ]:
data['enginePower'].sort_values().plot.hist(bins=20)

In [ ]:
np.log(data['enginePower'].sort_values()).plot.hist(bins=20)

In [ ]:
data['fuelType'] = data['fuelType'].astype('category')
data['fuelType'].value_counts().plot(kind='bar')

In [ ]:
data['productionDate'].value_counts()

In [ ]:
data['productionDate'].hist(bins=70)

In [ ]:
vehicle = {'AUTOMATIC':'автоматическая',
'MECHANICAL':'механическая',
'ROBOT':'роботизированная',
'VARIATOR':'вариатор'}
data['vehicleTransmission'] = data['vehicleTransmission'].replace(vehicle)

In [ ]:
data['vehicleTransmission'] = data['vehicleTransmission'].astype('category')
data['vehicleTransmission'].value_counts().plot.bar()


In [ ]:
data['vendor'] = data['vendor'].fillna('VENDOR_UNKNOWN')
data['vendor'].value_counts().plot.bar()
data['vendor'] = data['vendor'].astype('category')

In [ ]:
data['Владельцы'] = data['Владельцы'].fillna(data['Владельцы'].describe().top)
data['Владельцы'] = data['Владельцы'].apply(lambda x: int(re.findall('\d', str(x))[0]))
data['Владельцы'] = data['Владельцы'].astype('category')
data['Владельцы'].value_counts().plot(kind='bar')

In [ ]:
data['ПТС'].value_counts()

In [ ]:
pts = {'ORIGINAL':    'Оригинал',     'DUPLICATE':     'Дубликат'}       
data['ПТС'] = data['ПТС'].replace(pts)
data['ПТС'] = data['ПТС'].fillna(data['ПТС'].describe().top)
data['ПТС'].value_counts().plot(kind='bar')


In [ ]:
data['Привод'] = data['Привод'].astype('category')
data['Привод'].value_counts().plot(kind='bar')

In [ ]:
rule={
    'LEFT':      
'Левый',      
'RIGHT':       
'Правый'      
}
data['Руль'] = data['Руль'].replace(rule)
data['Руль'].value_counts().plot.bar()

In [ ]:
data['price'] = data['price'].fillna(0).astype('int')


In [ ]:
data[data['sample'] == 1]['price'].value_counts().plot.box()

In [ ]:
data['new_car'].value_counts().plot.bar()

In [ ]:
data['dont_use_car'].value_counts().plot.bar()

In [ ]:
data = data.drop(['model'], axis=1)

In [ ]:
data.info()

In [ ]:
data.drop(['log_price'], axis=1).corr()

productionDate - имеет высокую корреляцию с mileage, по этому мы удаляем этот столбец

In [ ]:
data = data.drop(['productionDate'], axis=1)

In [ ]:
for colum in ['bodyType', 'brand', 'color', 'engineDisplacement', 'fuelType', 'model_name', 'vehicleTransmission', 'vendor', 'Владельцы', 'Привод','ПТС','Руль']:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X = X[X['price'] != 0]  

In [ ]:
y_log = X['log_price']

In [ ]:
y = X['price']

X = X.drop(['price', 'log_price'], axis=1)
X_sub = data.query('sample == 0').drop(['sample','price', 'log_price'], axis=1)

In [ ]:
X_sub.info()

In [ ]:
X.info()

## Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
tmp_train = X_train.copy()



In [ ]:
tmp_train.head()

In [ ]:
y_train.head()

In [ ]:
X_test.head()

In [ ]:
y_test.head()

In [ ]:
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [ ]:
'''model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")
'''

Точность модели по метрике MAPE: 22.18%

Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 18%!

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [ ]:
'''model_log = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          use_best_model=True
                         )

model_log.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         plot=True
         )

model_log.save_model('catboost_single_model_2_baseline.model')
predict_test = np.exp(model_log.predict(X_test))
predict_submission_log = np.exp(model_log.predict(X_sub))
sample_submission['price'] = predict_submission_log
sample_submission.to_csv(f'submission_cat_log_v{VERSION}.csv', index=False)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")'''

Точность модели по метрике MAPE: 17.32%

Как видим точность возросла до 15%, а что будет на ЛБ? бейзлайн

In [ ]:
'''model10 = CatBoostRegressor(iterations = 10000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          use_best_model=True,
                          max_depth=7,
                            
                         )
model10.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         plot=True
         )

model10.save_model('catboost_single_model_2_baseline.model')
predict_test = np.exp(model10.predict(X_test))
predict_submission_log_10 = np.exp(model10.predict(X_sub))
sample_submission['price'] = predict_submission_log_10
sample_submission.to_csv(f'submission_cat_log10_v{VERSION}.csv', index=False)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")'''

Точность модели по метрике MAPE: 16.13%

In [ ]:
'''from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
estimators = [
    ('cb',model),
    ('rf',RandomForestRegressor(random_state=RANDOM_SEED))
]
reg = StackingRegressor(estimators=estimators)
reg.fit(X_train, np.log(y_train))
predict_test = reg.predict(X_test)
predict_submission = np.exp(reg.predict(X_sub))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_cat_stacR_v{VERSION}.csv', index=False)
predict_test = np.exp(predict_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")'''

Точность модели по метрике MAPE: 16.17%

In [ ]:
'''from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.003, max_depth=15, n_estimators=300)
gb.fit(X_train, np.log(y_train))
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(X_sub))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_gb_300_v{VERSION}.csv', index=False)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")'''


Точность модели по метрике MAPE: 43.54%

In [ ]:
'''from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.003, max_depth=15, n_estimators=600)
gb.fit(X_train, np.log(y_train))
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(X_sub))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_gb600_v{VERSION}.csv', index=False)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")'''

Точность модели по метрике MAPE: 24.62%

In [ ]:
'''from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.003, max_depth=15, n_estimators=1000)
gb.fit(X_train, np.log(y_train))
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(X_sub))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_gb_v{VERSION}.csv', index=False)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")'''

Точность модели по метрике MAPE: 18.02%

In [ ]:
'''from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.003, max_depth=15, n_estimators=5000)
gb.fit(X_train, np.log(y_train))
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(X_sub))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_gb_v{VERSION}.csv', index=False)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")'''

Точность модели по метрике MAPE: 15.85%

In [ ]:
'''from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.003, max_depth=15, n_estimators=10000)
gb.fit(X_train, np.log(y_train))
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(X_sub))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_gb10_v{VERSION}.csv', index=False)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")'''

Точность модели по метрике MAPE: 15.83%

In [ ]:
'''from sklearn.ensemble import GradientBoostingRegressor
gbc = GradientBoostingRegressor(random_state=RANDOM_SEED)
gbc.fit(X_train, np.log(y_train))
predict_test = np.exp(gbc.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")'''

Точность модели по метрике MAPE: 28.98%

In [ ]:
def generate_metafeatures(classifiers, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of classifiers.
    
    :arg classifiers: list of scikit-learn classifiers
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features using the classifier.
    
    :arg clf: scikit-learn classifier
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
cv = KFold(n_splits=10, shuffle=True)

stacked_features_train, stacked_features_test = generate_metafeatures([
    LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000),
    LogisticRegression(C=0.001, penalty='l2', solver='liblinear', max_iter=5000),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=300)
], X_train, X_test, y_train, cv)
total_features_train = np.hstack([X_train, stacked_features_train])
total_features_test = np.hstack([X_test, stacked_features_test])
clf = LogisticRegression(penalty='none', solver='lbfgs')
clf.fit(stacked_features_train, cover_y_train)
accuracy_score(clf.predict(stacked_features_test),y_test)
predict_test = clf.predict(stacked_features_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")


In [ ]:
predict_test = clf.predict(stacked_features_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
'''model = CatBoostRegressor(random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                         )

from sklearn.model_selection import GridSearchCV
param_grid = {'learning_rate':[ 0.1, 1], 
              'n_estimators':[100,500 ],
             'max_depth': [10,12,14]
             }
clf = GridSearchCV(estimator = model, param_grid = param_grid, scoring='neg_median_absolute_error', n_jobs=-1, cv=10)
clf.fit(X_train, np.log(y_train))
predict_test_clf = np.exp(clf.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")
'''

In [ ]:
predict_submission_clf1 = np.exp(clf.predict(X_sub))

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

In [ ]:
'''model = CatBoostRegressor(random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                         )

from sklearn.model_selection import GridSearchCV
param_grid = {'learning_rate':[0.00001, 0.0001,], 
              'n_estimators':[700 ],
             'max_depth': [5,7,9]
             }
clf = GridSearchCV(estimator = model, param_grid = param_grid, scoring='neg_median_absolute_error', n_jobs=-1, cv=10)
clf.fit(X_train, np.log(y_train))
predict_test_clf = np.exp(clf.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")
'''

In [ ]:
a = pd.DataFrame(model.feature_importances_)
a['name'] = X_train.columns.values
a.sort_values(by=0).plot.hist()

In [ ]:
a.sort_values(by=0, ascending=False).plot.bar(x='name')

# Submission

In [ ]:
sample_submission['price'] = predict_submission_clf
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)

В итоге получили **MAPE 27%** на ЛБ! на бейзлайне.. после - 21%

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing